## SIMPLE CHAR-RNN 

In [1]:
from __future__ import print_function
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
tf.set_random_seed(0)  
print ("TENSORFLOW VERSION IS %s" % (tf.__version__))

TENSORFLOW VERSION IS 1.0.1


## DEFINE TRAINING SEQUENCE

In [2]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")
print ("FOLLOWING IS OUR TRAINING SEQUENCE:")
print (sentence)

FOLLOWING IS OUR TRAINING SEQUENCE:
if you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.


## DEFINE VOCABULARY AND DICTIONARY

In [3]:
char_set = list(set(sentence))
char_dic = {w: i for i, w in enumerate(char_set)}
print ("VOCABULARY: ")
print (char_set)
print ("DICTIONARY: ")
print (char_dic)

VOCABULARY: 
[' ', "'", ',', '.', 'a', 'c', 'b', 'e', 'd', 'g', 'f', 'i', 'h', 'k', 'm', 'l', 'o', 'n', 'p', 's', 'r', 'u', 't', 'w', 'y']
DICTIONARY: 
{' ': 0, "'": 1, ',': 2, '.': 3, 'a': 4, 'c': 5, 'b': 6, 'e': 7, 'd': 8, 'g': 9, 'f': 10, 'i': 11, 'h': 12, 'k': 13, 'm': 14, 'l': 15, 'o': 16, 'n': 17, 'p': 18, 's': 19, 'r': 20, 'u': 21, 't': 22, 'w': 23, 'y': 24}


## CONFIGURE NETWORK

In [4]:
data_dim    = len(char_set)
num_classes = len(char_set)
hidden_size     = 64
sequence_length = 10  # Any arbitrary number

## SET TRAINING BATCHES

In [5]:
dataX = []
dataY = []
for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i + 1: i + sequence_length + 1]
    x = [char_dic[c] for c in x_str]  # x str to index
    y = [char_dic[c] for c in y_str]  # y str to index
    dataX.append(x)
    dataY.append(y)
    print ("[%3d/%3d] [%s]=>[%s]" % (i, len(sentence), x_str, y_str))
    print ("%s%s=>%s" % (' '*10, x, y))

[  0/180] [if you wan]=>[f you want]
          [11, 10, 0, 24, 16, 21, 0, 23, 4, 17]=>[10, 0, 24, 16, 21, 0, 23, 4, 17, 22]
[  1/180] [f you want]=>[ you want ]
          [10, 0, 24, 16, 21, 0, 23, 4, 17, 22]=>[0, 24, 16, 21, 0, 23, 4, 17, 22, 0]
[  2/180] [ you want ]=>[you want t]
          [0, 24, 16, 21, 0, 23, 4, 17, 22, 0]=>[24, 16, 21, 0, 23, 4, 17, 22, 0, 22]
[  3/180] [you want t]=>[ou want to]
          [24, 16, 21, 0, 23, 4, 17, 22, 0, 22]=>[16, 21, 0, 23, 4, 17, 22, 0, 22, 16]
[  4/180] [ou want to]=>[u want to ]
          [16, 21, 0, 23, 4, 17, 22, 0, 22, 16]=>[21, 0, 23, 4, 17, 22, 0, 22, 16, 0]
[  5/180] [u want to ]=>[ want to b]
          [21, 0, 23, 4, 17, 22, 0, 22, 16, 0]=>[0, 23, 4, 17, 22, 0, 22, 16, 0, 6]
[  6/180] [ want to b]=>[want to bu]
          [0, 23, 4, 17, 22, 0, 22, 16, 0, 6]=>[23, 4, 17, 22, 0, 22, 16, 0, 6, 21]
[  7/180] [want to bu]=>[ant to bui]
          [23, 4, 17, 22, 0, 22, 16, 0, 6, 21]=>[4, 17, 22, 0, 22, 16, 0, 6, 21, 11]
[  8/180] [ant to b

In [6]:
ndata      = len(dataX)
batch_size = 256
print ("     'NDATA' IS %d" % (ndata))
print ("'BATCH_SIZE' IS %d" % (batch_size))

     'NDATA' IS 170
'BATCH_SIZE' IS 256


## DEFINE PLACEHOLDERS

In [7]:
X = tf.placeholder(tf.int32, [None, sequence_length])
Y = tf.placeholder(tf.int32, [None, sequence_length])
X_OH = tf.one_hot(X, num_classes)
print ("'sequence_length' IS [%d]" % (sequence_length))
print ("    'num_classes' IS [%d]" % (num_classes))
print("'X' LOOKS LIKE \n   [%s]" % (X))  
print("'X_OH' LOOKS LIKE \n   [%s]" % (X_OH))

'sequence_length' IS [10]
    'num_classes' IS [25]
'X' LOOKS LIKE 
   [Tensor("Placeholder:0", shape=(?, 10), dtype=int32)]
'X_OH' LOOKS LIKE 
   [Tensor("one_hot:0", shape=(?, 10, 25), dtype=float32)]


## DEFINE MODEL

In [8]:
num_hidden = 128
with tf.variable_scope('CHAR-RNN', reuse=False):
    cell = rnn.BasicLSTMCell(hidden_size, state_is_tuple=True)
    cell = rnn.MultiRNNCell([cell] * 2, state_is_tuple=True)
    # DYNAMIC RNN WITH FULLY CONNECTED LAYER
    _hiddens  = tf.contrib.layers.fully_connected(X_OH, num_hidden, activation_fn=tf.nn.relu)
    _outputs, _states = tf.nn.dynamic_rnn(cell, _hiddens, dtype=tf.float32)
    _outputs  = tf.contrib.layers.fully_connected(_outputs, num_classes, activation_fn=None)
    # RESHAPE FOR SEQUNCE LOSS
    outputs = tf.reshape(_outputs, [batch_size, sequence_length, num_classes])
print ("OUTPUTS LOOKS LIKE [%s]" % (outputs))
print ("MODEL DEFINED.")

OUTPUTS LOOKS LIKE [Tensor("CHAR-RNN/Reshape:0", shape=(256, 10, 25), dtype=float32)]
MODEL DEFINED.


## DEFINE TF FUNCTIONS

In [9]:
# EQUAL WEIGHTS
weights = tf.ones([batch_size, sequence_length])
seq_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
loss = tf.reduce_mean(seq_loss)
optm  = tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss)
print ("FUNCTIONS DEFINED.")

FUNCTIONS DEFINED.


## OPTIMIZE

In [10]:
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())
MAXITER = 2000
for i in range(MAXITER):
    randidx = np.random.randint(low=0, high=ndata, size=batch_size)
    batchX = [dataX[iii] for iii in randidx]
    batchY = [dataY[iii] for iii in randidx]
    feeds = {X: batchX, Y: batchY}
    _, loss_val, results = sess.run(
        [optm, loss, outputs], feed_dict=feeds)
    if (i%100) == 0:
        print ("[%3d] loss_val: %.5f " % (i, loss_val))

[  0] loss_val: 3.21888 
[100] loss_val: 0.26556 
[200] loss_val: 0.24655 
[300] loss_val: 0.24302 
[400] loss_val: 0.25079 
[500] loss_val: 0.23487 
[600] loss_val: 0.22933 
[700] loss_val: 0.24101 
[800] loss_val: 0.23153 
[900] loss_val: 0.23398 
[1000] loss_val: 0.23909 
[1100] loss_val: 0.23926 
[1200] loss_val: 0.23536 
[1300] loss_val: 0.23715 
[1400] loss_val: 0.23741 
[1500] loss_val: 0.23405 
[1600] loss_val: 0.22429 
[1700] loss_val: 0.22781 
[1800] loss_val: 0.23368 
[1900] loss_val: 0.22227 


## PRINT CHARS

In [11]:
randidx = np.random.randint(low=0, high=ndata, size=batch_size)
batchX = [dataX[iii] for iii in randidx]
batchY = [dataY[iii] for iii in randidx]
feeds = {X: batchX, Y: batchY}
results = sess.run(outputs, feed_dict=feeds)
for j, result in enumerate(results):
    index = np.argmax(result, axis=1)
    chars = [char_set[t] for t in index]
    print (chars)

['e', 'm', ' ', 't', 'a', ' ', 'l', 'o', 'n', 'g']
['t', 'o', 'n', 'g', ' ', 'f', 'o', 'r', ' ', 't']
['h', 'e', 'm', 'e', 'n', 'd', 'l', 'e', 's', 's']
['t', 'h', 'i', 'p', ',', ' ', 'd', 'o', 'n', "'"]
['h', ' ', 'l', 'u', 'i', 'l', 'd', ' ', 'a', ' ']
[' ', 'a', 'o', 'r', 'k', ',', ' ', 'b', 'u', 't']
['l', ',', ' ', 'd', 'o', 'n', "'", 't', ' ', 'd']
[' ', 'i', 'n', 'd', ' ', 'w', 'o', 'r', 'k', ',']
['d', 'i', 't', 'y', ' ', 'o', 'f', ' ', 't', 'h']
['m', 'd', 'i', 't', 'y', ' ', 'o', 'f', ' ', 't']
['p', 'l', 'd', ' ', 'a', ' ', 's', 'h', 'i', 'p']
['h', 'l', 'l', 'e', 'c', 't', ' ', 'w', 'o', 'o']
[' ', 't', 'h', 'e', 'r', ' ', 't', 'e', 'a', 'c']
['e', 'm', ' ', 't', 'e', 'a', 'c', 'h', ' ', 't']
[' ', 'a', 'o', 'r', 'k', ',', ' ', 'b', 'u', 't']
['t', 'o', 'a', 'c', 'h', ' ', 't', 'h', 'e', 'm']
['s', ' ', 'b', 'u', 't', ' ', 'r', 'a', 't', 'h']
['e', 'p', ',', ' ', 'd', 'o', 'n', "'", 't', ' ']
[' ', 'd', ' ', 'a', 'n', 'd', ' ', 'd', 'o', 'n']
[' ', 'e', 't', 'h', 'e', 'r', 

### SAMPLING FUNCTION 

In [12]:
LEN = 1;
# XL = tf.placeholder(tf.int32, [None, LEN])
XL     = tf.placeholder(tf.int32, [None, 1])
XL_OH  = tf.one_hot(XL, num_classes)
with tf.variable_scope('CHAR-RNN', reuse=True):
    cell_L = rnn.BasicLSTMCell(hidden_size, state_is_tuple=True)
    cell_L = rnn.MultiRNNCell([cell_L] * 2, state_is_tuple=True)
    istate = cell_L.zero_state(batch_size=1, dtype=tf.float32)
    # DYNAMIC RNN WITH FULLY CONNECTED LAYER
    _hiddens  = tf.contrib.layers.fully_connected(XL_OH, num_hidden, activation_fn=tf.nn.relu)
    _outputs_L, states_L = tf.nn.dynamic_rnn(cell_L, _hiddens
                                , initial_state=istate, dtype=tf.float32)
    _outputs_L  = tf.contrib.layers.fully_connected(
        _outputs_L, num_classes, activation_fn=None)
    # RESHAPE FOR SEQUNCE LOSS
    outputs_L = tf.reshape(_outputs_L, [LEN, 1, num_classes])
print (XL)

def weighted_pick(weights):
    t = np.cumsum(weights)
    s = np.sum(weights)
    return(int(np.searchsorted(t, np.random.rand(1)*s)))
def softmax(x):
    alpha = 1
    e_x = np.exp(alpha*(x - np.max(x)))
    return e_x / np.sum(e_x) # only difference

Tensor("Placeholder_2:0", shape=(?, 1), dtype=int32)


## SAMPLE

### BURNIN

In [13]:
prime = "if you "
istateval = sess.run(cell_L.zero_state(1, tf.float32))
for c in prime[:-1]:
    index = char_dic[c]
    inval = [[index]]
    outval, stateval = sess.run([outputs_L, states_L]
                        , feed_dict={XL:inval, istate:istateval})
    istateval = stateval

### SAMPLE

In [14]:
inval  = [[char_dic[prime[-1]]]]
outval, stateval = sess.run([outputs_L, states_L]
                    , feed_dict={XL:inval, istate:istateval})
index = np.argmax(outval)
char  = char_set[index]
chars = ''
for i in range(500):
    inval = [[index]]
    outval, stateval = sess.run([outputs_L, states_L]
                        , feed_dict={XL:inval, istate:istateval})
    istateval = stateval
    # index = np.argmax(outval)
    index = weighted_pick(softmax(outval))
    char  = char_set[index]
    chars += char

print ("SAMPLED SETENCE: \n %s" % (prime+chars))
print ("\nORIGINAL SENTENCE: \n %s" % (sentence))

SAMPLED SETENCE: 
 if you ant to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.h them to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.h them to long for the endless immensity of the sea.h them to long for the endless immensity of the sea.h them to long for the endless immensity of the sea.h them to long for the endless immensity of 

ORIGINAL SENTENCE: 
 if you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.


## CLOSE SESSION

In [15]:
sess.close()
print ("SESSION CLOSED.")

SESSION CLOSED.
